# https://github.com/LanLi2017/OpenRefine_client_Tutorial#openrefine-python-client-library

To ask for communicating with OpenRefine server.
Notes:
1. Ensure you have OpenRefine.[http://127.0.0.1:3333/] running 
2. Server and Project id are required for sending request to OpenRefine server.
3. OR.py has encapsulated and extended functions based on OR-Client library .[https://github.com/LanLi2017/OpenRefineClientPy3] 
4. After applying each operation, a 'history id' will be generated and corresponding history meta-info will be Auto-stored in your local machine. You might check it from worksapce. 
5. update: crsf_token is needed when you send request, OpenRefine 3.3+
6. Distinguish 'operation history' and 'history'
7. Undo/Redo: if you want to undo some step, e.g step 3, apply the history id of previous step which is history id of step 2; if you want to redo some step, apply the history id of this step, which is history id of step 3.  

In [1]:
from OpenRefineClientPy3.google_refine.refine import refine
from OR import Refineop
import pandas as pd
import json
from pprint import pprint

# Project

In [2]:
# use refine.py to directly connect with server
projects = refine.Refine(refine.RefineServer()).list_projects()

In [3]:
# use OR.py to connect with server 
# more functions supported, and refined outputs
# Params: (server, project id)
server = refine.RefineServer()
projectID = 2014260363969
refineop = Refineop(server,projectID)

In [4]:
# List history (From internal package)
# Output: 'past' stands for current operation history; 'future' stands for 'undo' part 
# Description+id+time: metadata for each data cleaning step
history_id = refineop.list_history()
pprint(history_id)
hid = [hid['id'] for hid in history_id['past']]
print(hid)

{'future': [{'description': 'Text transform on 49 cells in column Posts: '
                            'value.toNumber()',
             'id': 1587412238868,
             'time': '2020-04-20T19:47:19Z'},
            {'description': 'Text transform on 0 cells in column Author_Name: '
                            'value.toUppercase()',
             'id': 1587412342560,
             'time': '2020-04-20T19:47:58Z'}],
 'past': [{'description': 'Text transform on 49 cells in column Date (CST): '
                          'value.toDate()',
           'id': 1587369173241,
           'time': '2020-04-20T07:42:09Z'},
          {'description': 'Text transform on 49 cells in column Author: '
                          'value.replace("?"," ")',
           'id': 1587412700412,
           'time': '2020-04-20T19:42:46Z'},
          {'description': 'Create new column Author_Name based on column '
                          'Author by filling 49 rows with '
                          "grel:cells.Author.value

In [5]:
# Redo step 3: hid ==
# Params: history id for step 2 (3-1)
refinesever =refine.RefineServer()
csrf_t = refinesever.get_csrf()['token']
refineop.undo_redo_proj(lastDone_id = 1587412161715, token = csrf_t)

True

In [6]:
# Check if we've successfully redo step 3
# Return: past add up one step; future minus one step 
history_id = refineop.list_history()
pprint(history_id)

{'future': [{'description': 'Text transform on 49 cells in column Posts: '
                            'value.toNumber()',
             'id': 1587412238868,
             'time': '2020-04-20T19:47:19Z'},
            {'description': 'Text transform on 0 cells in column Author_Name: '
                            'value.toUppercase()',
             'id': 1587412342560,
             'time': '2020-04-20T19:47:58Z'}],
 'past': [{'description': 'Text transform on 49 cells in column Date (CST): '
                          'value.toDate()',
           'id': 1587369173241,
           'time': '2020-04-20T07:42:09Z'},
          {'description': 'Text transform on 49 cells in column Author: '
                          'value.replace("?"," ")',
           'id': 1587412700412,
           'time': '2020-04-20T19:42:46Z'},
          {'description': 'Create new column Author_Name based on column '
                          'Author by filling 49 rows with '
                          "grel:cells.Author.value

In [7]:
# List operation history 
operation_history = refineop.get_operations()
pprint(operation_history)

[{'columnName': 'Date (CST)',
  'description': 'Text transform on cells in column Date (CST) using '
                 'expression value.toDate()',
  'engineConfig': {'facets': [], 'mode': 'row-based'},
  'expression': 'value.toDate()',
  'onError': 'keep-original',
  'op': 'core/text-transform',
  'repeat': False,
  'repeatCount': 10},
 {'columnName': 'Author',
  'description': 'Text transform on cells in column Author using expression '
                 'value.replace("?"," ")',
  'engineConfig': {'facets': [], 'mode': 'row-based'},
  'expression': 'value.replace("?"," ")',
  'onError': 'keep-original',
  'op': 'core/text-transform',
  'repeat': False,
  'repeatCount': 10},
 {'baseColumnName': 'Author',
  'columnInsertIndex': 5,
  'description': 'Create column Author_Name at index 5 based on column Author '
                 'using expression '
                 "grel:cells.Author.value+'-'+cells.Name.value",
  'engineConfig': {'facets': [], 'mode': 'row-based'},
  'expression': "grel:c

Difference between operation history and history!
1. Content: operation history returns prospective provenance; history returns retrospective provenance
2. Operation history only returns "past"; history returns "past" and "future"

# Facet

# Cell (Inner-Column)

In [8]:
# Text Transform 
# Params: column name, expression, csrf required
# Return: status, history

refinesever =refine.RefineServer()
csrf_t = refinesever.get_csrf()['token']

refineop.text_transform('Author',expression = 'value.toUppercase()', token= csrf_t)

{'historyEntry': {'id': 1605581715100,
  'description': 'Text transform on 0 cells in column Author: value.toUppercase()',
  'time': '2020-11-17T02:44:03Z'},
 'code': 'ok'}

In [9]:
from collections import OrderedDict
# Cluter and Edit 
# 1. compute clusters: return a list of clusters; 
#    params: column, clusterer_type='binning', function=None, params=None
# Mass_edit
# 2. apply (edits_from, edits_to) 
# Return: status, history

clusters = refineop.compute_clusters('Country', clusterer_type='binning', function='fingerprint')
print(clusters)

# choose cluster
from_edit=refineop.getFromValue(clusters)
to_edit=refineop.getToValue(clusters)
default_edits=OrderedDict()
default_edits['fromBlank']='false'
default_edits['fromError']='false'
for f1,t1 in zip(from_edit, to_edit):
    default_edits['from']=f1
    default_edits['to']=t1
edits=[default_edits]
# mass edit : column, edits
refineop.mass_edit('Country', edits)

[]


Exception: server error: Missing or invalid csrf_token parameter

In [17]:
# Check operation history
pprint(refineop.get_operations())

[{'columnName': 'Date (CST)',
  'description': 'Text transform on cells in column Date (CST) using '
                 'expression value.toDate()',
  'engineConfig': {'facets': [], 'mode': 'row-based'},
  'expression': 'value.toDate()',
  'onError': 'keep-original',
  'op': 'core/text-transform',
  'repeat': False,
  'repeatCount': 10},
 {'columnName': 'Author',
  'description': 'Text transform on cells in column Author using expression '
                 'value.replace("@"," ")',
  'engineConfig': {'facets': [], 'mode': 'row-based'},
  'expression': 'value.replace("@"," ")',
  'onError': 'keep-original',
  'op': 'core/text-transform',
  'repeat': False,
  'repeatCount': 10},
 {'columnName': 'Author',
  'description': 'Text transform on cells in column Author using expression '
                 'value.trim()',
  'engineConfig': {'facets': [], 'mode': 'row-based'},
  'expression': 'value.trim()',
  'onError': 'keep-original',
  'op': 'core/text-transform',
  'repeat': False,
  'repeatCou

In [18]:
# Single edit (27,5) from (@KIMBA707) to (KIMBA707)
# Params: row, cell, type, value
# is there any ops recorded?
refineop.single_edit(27,5,'text','KIMBA707')

{'code': 'ok',
 'historyEntry': {'id': 1587681736726,
  'description': 'Edit single cell on row 28, column Name',
  'time': '2020-04-23T22:26:16Z'},
 'cell': {'v': 'KIMBA707'},
 'pool': {'recons': {}}}

In [19]:
# Check history 

cur_history = refineop.list_history()
pprint(cur_history)

{'future': [],
 'past': [{'description': 'Text transform on 1000 cells in column Date (CST): '
                          'value.toDate()',
           'id': 1587318240278,
           'time': '2020-04-19T17:38:16Z'},
          {'description': 'Text transform on 1000 cells in column Author: '
                          'value.replace("@"," ")',
           'id': 1587318248763,
           'time': '2020-04-19T17:38:16Z'},
          {'description': 'Text transform on 1000 cells in column Author: '
                          'value.trim()',
           'id': 1587318074136,
           'time': '2020-04-19T17:38:16Z'},
          {'description': 'Text transform on 976 cells in column Author: '
                          'value.toUppercase()',
           'id': 1587681004170,
           'time': '2020-04-23T22:23:42Z'},
          {'description': 'Mass edit 2 cells in column Country',
           'id': 1587680789392,
           'time': '2020-04-23T22:25:06Z'},
          {'description': 'Edit single cell on

In [20]:
# Check operation history
pprint(refineop.get_operations())

[{'columnName': 'Date (CST)',
  'description': 'Text transform on cells in column Date (CST) using '
                 'expression value.toDate()',
  'engineConfig': {'facets': [], 'mode': 'row-based'},
  'expression': 'value.toDate()',
  'onError': 'keep-original',
  'op': 'core/text-transform',
  'repeat': False,
  'repeatCount': 10},
 {'columnName': 'Author',
  'description': 'Text transform on cells in column Author using expression '
                 'value.replace("@"," ")',
  'engineConfig': {'facets': [], 'mode': 'row-based'},
  'expression': 'value.replace("@"," ")',
  'onError': 'keep-original',
  'op': 'core/text-transform',
  'repeat': False,
  'repeatCount': 10},
 {'columnName': 'Author',
  'description': 'Text transform on cells in column Author using expression '
                 'value.trim()',
  'engineConfig': {'facets': [], 'mode': 'row-based'},
  'expression': 'value.trim()',
  'onError': 'keep-original',
  'op': 'core/text-transform',
  'repeat': False,
  'repeatCou

Difference between Operation History at client end and Operation History at server end

# Column

In [21]:
# Addition Column
# Params: column, new_column, expression='value', column_insert_index=None
# Addition expression could be found https://github.com/OpenRefine/OpenRefine/wiki/Column-Editing
refineop.add_column('Country','Country_State', expression='cells.Country.value+cells.State.value')

{'code': 'ok',
 'historyEntry': {'id': 1587681471431,
  'description': 'Create new column Country_State based on column Country by filling 0 rows with cells.Country.value+cells.State.value',
  'time': '2020-04-23T22:27:56Z'}}

In [22]:
# Split Column
# Params: column, separator=','
refineop.split_column('Name',' ')

{'code': 'ok',
 'historyEntry': {'id': 1587681204166,
  'description': 'Split 1000 cell(s) in column Name into several columns by separator',
  'time': '2020-04-23T22:28:18Z'}}

In [23]:
# Rename Column
# Params: old column name, new column name
refineop.rename_column('Name 1', 'FirstName')

{'code': 'ok',
 'historyEntry': {'id': 1587681205870,
  'description': 'Rename column Name 1 to FirstName',
  'time': '2020-04-23T22:28:31Z'}}

# Row level

In [24]:
# Annotate Row (star/flag)
# Row index, flagged:default='True'
refineop.flag_row(1)
refineop.star_row(2)

{'code': 'ok',
 'historyEntry': {'id': 1587681004243,
  'description': 'Star row 3',
  'time': '2020-04-23T22:28:44Z'}}

# Download Operation History 

In [25]:
# Load operation history
# Params: load file name
# Return: auto load operation history in your local machine
file_name = 'load_recipe'
refineop.load_ops(file_name)

# Download Cleaned Dataset

In [26]:
# Load cleaned dataset
# params: file name
# return: auto load dataset in your local machine
filename = 'CleanedData.csv'
refineop.load_data(filename)

check https://github.com/tmcphillips/openrefine-provenance/blob/master/models/openrefine%20data%20and%20project%20model.md for OpenRefine taxonomy in OpenRefine